# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "data/train.p"
testing_file = "data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.
import numpy as np

# TODO: Number of training examples
n_train = y_train.shape[0]

# TODO: Number of testing examples.
n_test = y_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
matplotlib.style.use('ggplot')
# Visualizations will be shown in the notebook.
%matplotlib inline
import pandas as pd
from math import ceil
import random

In [ ]:
# Sample images
def show_images_in_grid(*img_keys, cols=16, fig_width=12.0, X_data=None, y_data=None):
    X_data = X_data if X_data is not None else X_train
    y_data = y_data if y_data is not None else y_train
    
    img_count = len(img_keys)
    rows = ceil(img_count / cols)
    fig_height = (fig_width * rows / cols) + 0.6
    
    grid = gridspec.GridSpec(rows, cols, wspace=0.0, hspace=0.2)
    fig = plt.figure(figsize=(fig_width, fig_height))
    for i, m in enumerate(img_keys):
        ax = plt.Subplot(fig, grid[i])
        img = X_data[m]
        if img.shape[-1] > 1:
            ax.imshow(X_data[m], interpolation='bicubic')
        else:
            ax.imshow(X_data[m,:,:,0], cmap='gray', interpolation='bicubic')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title("Class: {}".format(y_data[m]), size=6)
        fig.add_subplot(ax)
        
    plt.show()

In [ ]:
print("Sample images:")
sample_idx = random.sample(range(n_train), 48)
show_images_in_grid(*sample_idx)

In [ ]:
# Class frequency distribution
def show_distributions(labels, set_name, title="Class Frequency Distribution"):
    data = {}
    for name, y in zip(set_name, labels):
        idx, freqs = np.unique(y, return_counts=True)
        series = pd.Series(freqs, index=idx)
        data[name] = series
      
    df = pd.DataFrame(data)
    df.index.name = "Classes"
    ag = df.plot.bar(figsize=(15, 5), title=title)
    ag.set_ylabel("Count")
    plt.show()
    
show_distributions((y_train, y_test), ('Train', 'Test'))

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

In [ ]:
def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)[:,:,0]
    img = np.asarray(img, dtype=np.float32).reshape(img.shape + (1,))
    return (img - 128.) / 128.

if X_train.shape[-1] > 1:
    X_train = np.asarray([preprocess_image(m) for m in X_train], dtype=np.float32)
    X_test = np.asarray([preprocess_image(m) for m in X_test], dtype=np.float32)
    
print("X_train: shape: {}, min:{}, max: {}".format(X_train.shape, X_train.min(), X_train.max()))
print("X_test: shape: {}, min:{}, max: {}".format(X_test.shape, X_test.min(), X_test.max()))

print("\nSample preprocessed images:")
show_images_in_grid(*sample_idx)

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

Traffic signs are generally designed to be legible. The signs appear to still be readable even without colors. The images are converted to gray scale by first converting the color space from RGB to YUV and keep the Y channel only. This reduces the feature size to 1/3 of the original size while remaining to have meaningful images.

The values are shifted to zero center and scaled in the range of -1 to 1 which will help in the learning process.

In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

def stratified_train_val_split(labels, train=0.8, seed=None):
    """Stratify split data into training-validation sets"""

    saved_state = np.random.get_state()
    np.random.seed(seed)

    y = np.asarray(labels)
    train_idx = np.zeros(y.shape[0], dtype=np.bool)
    val_idx = np.zeros(y.shape[0], dtype=np.bool)
    classes = np.unique(y)
    for cls in classes:
        idx = np.nonzero(y == cls)[0]
        np.random.shuffle(idx)
        n = int(train * idx.shape[0])

        train_idx[idx[:n]] = True
        val_idx[idx[n:]] = True
    
    t_idx = np.nonzero(train_idx)[0]
    np.random.shuffle(t_idx)
    v_idx = np.nonzero(val_idx)[0]
    np.random.shuffle(v_idx)
    
    np.random.set_state(saved_state)
    return t_idx, v_idx

def split_data(X, y, train=0.7, seed=None):
    t_idx, v_idx = stratified_train_val_split(y, train=train, seed=seed)
    return X[t_idx], y[t_idx], X[v_idx], y[v_idx]


# Split training dataset to 70-30 training-validation set
if X_train.shape[0] >= n_train:
    X_train, y_train, X_val, y_val = split_data(X_train, y_train, train=0.8)
    
print("Number of training samples =", y_train.shape[0])
print("Number of validation samples =", y_val.shape[0])
print("Number of testing samples =", y_test.shape[0])
show_distributions((y_train, y_val, y_test), ('Train', 'Validation', 'Test'))

In [ ]:
def random_transform(img, rotate=10.0, translate=5, perpective=5):
    orig_rows, orig_cols = img.shape[:2]
    
    # pad with zeros to avoid clipping
    border = img.shape[0] // 2
    img = cv2.copyMakeBorder(img, border, border, border, border, cv2.BORDER_CONSTANT, value=-1.0) 
    
    
    # rotate
    rows, cols = img.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 
                                np.random.uniform(low=-rotate, high=rotate),
                                1)
    img = cv2.warpAffine(img, M, (cols, rows))
    
    
    # translate
    dx, dy = np.random.uniform(low=-translate, high=translate, size=2)
    M = np.float32([[1, 0, dx],
                    [0, 1, dy]])
    img = cv2.warpAffine(img, M, (cols, rows))
    
    
    # perspective
    zero = border
    x00, y00 = zero            , zero
    x10, y10 = zero + orig_cols, zero
    x01, y01 = zero            , zero + orig_rows
    x11, y11 = zero + orig_cols, zero + orig_rows
    
    pts1 = np.float32([[x00, y00], [x10, y10], 
                       [x01, y01], [x11, y11]])
    
    dx1, dx2, dy1, dy2 = np.random.uniform(low=-perpective, high=perpective, size=4)
    
    if np.random.uniform() >= 0.5:
        # vertical perspective
        pts2 = np.float32([[x00 + dx1, y00 + dy1], [x10 - dx1, y10 + dy1], 
                           [x01 + dx2, y01 + dy2], [x11 - dx2, y11 + dy2]])
    else:
        # horizontal perspective
        pts2 = np.float32([[x00 + dx1, y00 + dy1], [x10 + dx2, y10 + dy2], 
                           [x01 + dx1, y01 - dy1], [x11 + dx2, y11 - dy2]])
    
    M = cv2.getPerspectiveTransform(pts1, pts2)
    img = cv2.warpPerspective(img, M, (cols, rows))
    
    return img[border:border + orig_rows, border:border + orig_cols]

def synthesize_data(X, target=None):
    count, rows, cols = X.shape[:3]
    if target is None:
        if count < 200: target = count * 3
        elif count < 400: target = count * 2
        elif count < 800: target = count * 1
        else: return X
    else:
        target -= count
    if target <= 0: return X
    
    trans = lambda i: random_transform(X[i % count]).reshape((rows, cols, 1))
    synth = np.asarray([trans(i) for i in range(target)])
    return np.concatenate((X, synth), axis=0)

def generate_synthetic_data(X, y, target=None):
    classes = np.unique(y)
    X_temp, y_temp = [], []
    
    for cls in classes:
        idx = np.nonzero(y == cls)[0]
        imgs = X[idx]
        synths = synthesize_data(imgs, target=target)
        X_temp.extend(synths)
        y_temp.extend([cls] * synths.shape[0])
    
    return np.asarray(X_temp), np.asarray(y_temp)

In [ ]:
def show_sample_synthesized_images(img, label):
    imgs = np.array([img])
    synths = synthesize_data(imgs, target=48)
    sample_idx = range(synths.shape[0])
    
    print("Sample synthesized images (upper leftmost image is the original reference image):")
    show_images_in_grid(*sample_idx, X_data=synths, y_data=[label] * synths.shape[0])

show_sample_synthesized_images(X_val[11], y_val[11])

In [ ]:
# Generate synthetic data
X_train, y_train = generate_synthetic_data(X_train, y_train)

print("Number of training samples =", y_train.shape[0])
print("Number of validation samples =", y_val.shape[0])
print("Number of testing samples =", y_test.shape[0])
show_distributions((y_train, y_val, y_test), ('Train', 'Validation', 'Test'))

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import os
import tensorflow as tf

In [ ]:
EPOCHS = 10

TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128

IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
CHANNELS = 1

LABELS_COUNT = n_classes

LEARNING_RATE = 1e-3
DECAY_STEP = X_train.shape[0] // TRAIN_BATCH_SIZE
LEARNING_RATE_DECAY = 0.98

CHECK_POINT_DIR = "models"
if not os.path.isdir(CHECK_POINT_DIR): os.makedirs(CHECK_POINT_DIR)
CHECK_POINT_FILE = os.path.join(CHECK_POINT_DIR, "model.ckpt")
CHECK_POINTS_TO_KEEP = 5

LOG_AT_EVERY = 100

In [ ]:
##### def make_model(X):    
#     # Stage 1
#     # Input = 32x32x1. Output = 16x16x8
#     conv1 = make_conv(X, kernel=5, depth=8)
#     conv1 = max_pool(conv1, kernel=2, stride=2)
    
#     # Stage 2
#     # Input = 16x16x8. Output = 8x8x16
#     conv2 = make_conv(conv1, kernel=3, depth=16)
#     conv2 = max_pool(conv2, kernel=2, stride=2)
    
#     # Classifier
#     conv1_flat = tf.contrib.layers.flatten(conv1)
#     conv2_flat = tf.contrib.layers.flatten(conv2)
    
#     flat = tf.concat(1, [conv1_flat, conv2_flat])
#     logits = make_fc(flat, LABELS_COUNT)
    
#     return logits

def make_model(X, keep_prob):    
    # Stage 1
    # Input = 32x32x1
    conv1 = make_conv(X, kernel=5, depth=8, pad='VALID')     # 28x28x8
    conv1 = max_pool(conv1, kernel=2, stride=2, pad='VALID') # 14x14x8
    
    # Stage 2
    # Input = 16x16x32. Output = 8x8x64
    conv2 = make_conv(conv1, kernel=5, depth=16, pad='VALID') # 10x10x16
    conv2 = max_pool(conv2, kernel=2, stride=2, pad='VALID')  # 5x5x16
    
    # Dense Layer
    conv1_2 = max_pool(conv1, kernel=2, stride=2)   # 7x7x8
    conv1_flat = tf.contrib.layers.flatten(conv1_2) # 7*7*3 = 392
    conv2_flat = tf.contrib.layers.flatten(conv2)   # 5*5*16 = 400
    
    flat = tf.concat(1, [conv1_flat, conv2_flat])   # 392 + 400 = 792
    
    # Hidden Layers
    fc1 = make_fc(conv2_flat, 512)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    fc2 = make_fc(fc1, 256)
    fc2 = tf.nn.dropout(fc2, keep_prob)
    
    # Classifier
    logits = make_fc(fc2, LABELS_COUNT, activatation=False)

    return logits

# def test_model():
#     tf_X = tf.placeholder(tf.float32, 
#                                 shape=(None,
#                                        IMAGE_HEIGHT, IMAGE_WIDTH,
#                                        CHANNELS))
#     make_model(tf_X)
# test_model()

# def make_model(X):
#     # Layer 1: Convolution. Input = 32x32x1
#     conv1 = make_conv(X, kernel=5, depth=6, pad='VALID')     # 28x28x6
#     conv1 = max_pool(conv1, kernel=2, stride=2, pad='VALID') # 14x14x6

#     # Layer 2:
#     conv2 = make_conv(conv1, kernel=5, depth=16, pad='VALID')  # 10x10x16
#     conv2 = max_pool(conv2, kernel=2, stride=2, pad='VALID')   # 5x5x16
 
#     # Flatten:
#     fc0 = tf.contrib.layers.flatten(conv2)   # 5*5*16 = 400
    
#     # Layer 3: Fully Conected
#     fc1 = make_fc(fc0, 120)    # 120    

#     # Layer 4: Fully Connected
#     fc2 = make_fc(fc1, 84)    # 84 
    
#     # Logits
#     logits = make_fc(fc2, LABELS_COUNT, activatation=False)
    
#     return logits

def make_conv(in_data, kernel, depth, 
              stride=1, pad='SAME', std=0.1):
    
    in_depth = in_data.get_shape().as_list()[3]
    
    W = tf.Variable(tf.truncated_normal((kernel, kernel, in_depth, depth), 
                                         stddev=std))
    b = tf.Variable(tf.zeros(depth))
    strides = [1, stride, stride, 1]
    
    conv = tf.nn.conv2d(in_data, W, strides, padding=pad) + b
    
    return tf.nn.relu(conv)

def max_pool(in_data, kernel=2, stride=2, pad='SAME'):
    return tf.nn.max_pool(in_data,
                          ksize=[1, kernel, kernel, 1],
                          strides=[1, stride, stride, 1],
                          padding=pad)

# Make fully connected
def make_fc(in_data, out_c, std=0.1, activatation=True):
    # in_data expects a 2 dimensional tensor with shape [N, D]
    features = in_data.get_shape().as_list()[1]
    
    W = tf.Variable(tf.truncated_normal((features, out_c), stddev=std))
    b = tf.Variable(tf.zeros(out_c))
    
    fc = tf.matmul(in_data, W) + b

    return fc if not activatation else tf.nn.relu(fc)
    
def make_loss(logits, target):
    cross_en = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, target)
    return tf.reduce_mean(cross_en)

def optimize(loss):
    """Returns tensor for the optimizer for training"""
    
    global_step = tf.Variable(0, trainable=False)
    # Learning rate
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE, global_step, 
        DECAY_STEP,
        LEARNING_RATE_DECAY)
    
    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    return optimizer.minimize(loss, global_step=global_step)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    # Train input
    tf_X = tf.placeholder(tf.float32, 
                                shape=(None,
                                       IMAGE_HEIGHT, IMAGE_WIDTH,
                                       CHANNELS))
    tf_y = tf.placeholder(tf.int32, shape=(None,))
    
    tf_keep_prob = tf.placeholder(tf.float32)
    
    logits = make_model(tf_X, tf_keep_prob)
    loss = make_loss(logits, tf_y)
    optimizer = optimize(loss)
    
    prediction = tf.argmax(logits, 1)
    correct_prediction = tf.equal(prediction, tf.cast(tf_y, tf.int64))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    param_saver = tf.train.Saver(max_to_keep=CHECK_POINTS_TO_KEEP)

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

def batches(X, y=None, batch_size=None, epochs=1, trim_excess=False, shuffle=False):
    idx = np.arange(X.shape[0])
    batch_size = batch_size if batch_size is not None else X.shape[0]
    steps = X.shape[0] // batch_size
    if not trim_excess and X.shape[0] % batch_size != 0:
        steps += 1
    
    for i in range(epochs):
        if shuffle: np.random.shuffle(idx)
        for c in range(steps):
            offset = c * batch_size
            batch_idx = idx[offset:offset + batch_size]
            
            if y is None:
                yield X[batch_idx]
            else:
                yield X[batch_idx], y[batch_idx]
        
def do_train(session, X, y, validation):
    cp = tf.train.latest_checkpoint(CHECK_POINT_DIR)
    if cp is None:
        session.run(tf.global_variables_initializer())
        print("Initialized variables.")
    else:
        param_saver.restore(session, cp)
        print("Restored varaibles from:", cp)
    
    #################
    # TODO: 
    progress_file = "runs/train_progress.p"
    if os.path.isfile(progress_file):
        with open(progress_file, mode='rb') as f:
            losses, train_accuracies, val_accuracies = pickle.load(f)
        start_step = len(losses)
        val_acc = val_accuracies[-1]
        print("Restored stats from:", progress_file)
    else:
        losses, train_accuracies, val_accuracies = [], [], []
        start_step = 0
        print("Initialized empty stats.")
        
    
    batch = batches(X, y, TRAIN_BATCH_SIZE, EPOCHS, trim_excess=True, shuffle=True)
    for i, (batch_X, batch_y) in enumerate(batch):
        step = start_step + i
        feed_dict = {tf_X: batch_X, tf_y: batch_y, tf_keep_prob: 0.5}
        _, _loss, _acc = session.run([optimizer, loss, accuracy_op], feed_dict=feed_dict)
        losses.append(_loss)
        
        if step % LOG_AT_EVERY == 0:
            param_saver.save(session, CHECK_POINT_FILE, global_step=step)
            train_accuracies.append(_acc)
            val_acc = validation(session)
            val_accuracies.append(val_acc)
            
            #########################
            # TODO: Delete this block
            _save_progress(losses, train_accuracies, val_accuracies)
            #########################
        
        if step % (X.shape[0] // TRAIN_BATCH_SIZE) == 0:
            print("[Step: {}] Train loss: {}, Train acc: {}, Validation acc: {}"
                  .format(step, _loss, _acc, val_acc))
    
    param_saver.save(session, CHECK_POINT_FILE, global_step=step)
    _save_progress(losses, train_accuracies, val_accuracies) # TODO: Delete this
    return losses, train_accuracies, val_accuracies

# TODO: Delete this function
def _save_progress(losses, train_accuracies, val_accuracies):
    with open("runs/train_progress.p", mode="wb") as f:
        pickle.dump((losses, train_accuracies, val_accuracies), f)
    
def validate(X, y):
    def do_validation(session):
        preds = []
        batch = batches(X, y, VAL_BATCH_SIZE)
        for batch_X, batch_y in batch:
            feed_dict={tf_X: batch_X, tf_y: batch_y, tf_keep_prob: 1.0}
            preds.extend(session.run(correct_prediction, feed_dict=feed_dict))
        return np.mean(preds)
    return do_validation 

In [ ]:
# NOTE: Running this cell will take a while

def train():
    with tf.Session(graph=graph) as sess:
        validation = validate(X_val, y_val)
        loss, train_acc, valid_acc = do_train(sess, X_train, y_train, validation)
    return loss, train_acc, valid_acc

train_losses, train_accuracies, validation_accuracies = train()

In [ ]:
def plot_train_loss(loss):
    loss = pd.Series(loss)
    loss.index.name = "Iteration"
    ag = loss.plot(figsize=(15, 5), title="Training Loss")
    ag.set_ylabel("Loss")
    plt.show()

plot_train_loss(train_losses)

def plot_accuracies(accuracies):
    df = pd.DataFrame(accuracies)
    df['Iteration'] = ([i * LOG_AT_EVERY for i in df.index])
    df.set_index('Iteration', inplace=True)
    ag = df.plot(figsize=(15, 5), title="Accuracies")
    ag.set_ylabel("Accuracy")
    plt.show()
    
plot_accuracies({"Training": train_accuracies,
                 "Validation": validation_accuracies})

In [ ]:
def predict(X, batch_size=None):
    with tf.Session(graph=graph) as session:
        param_saver.restore(session, tf.train.latest_checkpoint(CHECK_POINT_DIR))
        
        preds = []
        batch = batches(X, batch_size=batch_size)
        for batch_X in batch:
            feed_dict={tf_X: batch_X, tf_keep_prob: 1.0}
            preds.extend(session.run(prediction, feed_dict=feed_dict))
    return np.asarray(preds)

In [ ]:
def predict_on_test_dataset():
    predictions = predict(X_test, batch_size=TEST_BATCH_SIZE)
    accuracy = np.mean(predictions == y_test)
    print("Test dataset accuracy:", accuracy)
predict_on_test_dataset()

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.